# NN Learning MNIST in Google Colab

In [2]:
import os
from pathlib import Path
import json
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

import matplotlib.pyplot as plt
import numpy as np

## Set Directories

In [12]:
!pwd

/home/sam/Projects/DSR_Batch30/notebooks/Tensorflow


In [14]:
project_path = "../../models/tf_colab_mnist/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

project_path = "/content/drive/MyDrive/Projects/DSR_Batch30/models/tf_colab_mnist"
!ls $project_path

## Load Data

In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# Onehot-Encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## Set Hyperparameters

In [4]:
session_name = "test1"

layer_counts = [1, 2]
layer_neuron_counts = [16, 32]
layer_activation_types = ["sigmoid", "relu"] #, "linear", "tanh"]

epochs = 3

## Create Model

In [5]:
def create_model(layer_count, layer_neuron_count, layer_activation):
    model = models.Sequential()
    
    model.add(layers.Flatten(input_shape=(28,28)))
    
    for n in range(layer_count):
        model.add(layers.Dense(layer_neuron_count, activation=layer_activation))
    
    model.add(layers.Dense(10, activation="softmax"))
    
    model.compile(
        loss="categorical_crossentropy",
        metrics=["accuracy"],
        optimizer="adam"
    )
    
    return model
        

In [6]:
def create_model_dir(project_path, session_name, model_name):
    session_dir = project_path+ session_name + "/"  
    model_dir = session_dir + model_name + "/"

    saved_dir = model_dir + "saved/"
    checkpoint_dir = model_dir + "cp/"
    history_dir = model_dir + "history/"

    checkpoint_path = checkpoint_dir + "cp-{epoch:02d}.ckpt"

    Path(saved_dir).mkdir(parents=True, exist_ok=True)
    Path(checkpoint_dir).mkdir(parents=True, exist_ok=True)
    Path(history_dir).mkdir(parents=True, exist_ok=True)
    
    return saved_dir, checkpoint_dir, history_dir, session_dir

In [7]:
stat_columns = ["model_name", "layers", "neurons", "activation", "epoch", "loss", "accuracy", "val_loss", "val_accuracy"]
session_stats = pd.DataFrame(columns=stat_columns)

In [10]:
for layer_count in layer_counts:
    for layer_neuron_count in layer_neuron_counts:
        for layer_activation in layer_activation_types:
            print()
            model_name = "_".join([session_name, "L" + str(layer_count), "N" + str(layer_neuron_count), layer_activation])
            print(model_name)
            
            model = create_model(layer_count, layer_neuron_count, layer_activation)
            model.summary()
            
            saved_dir, checkpoint_dir, history_dir, session_dir = create_model_dir(project_path, session_name, model_name)
            
            checkpoint_path = checkpoint_dir + "cp-{epoch:02d}.ckpt" 
            cp_callback = tf.keras.callbacks.ModelCheckpoint(
                filepath=checkpoint_path,
                save_weights_only=True,
                verbose=1
            )
            
            history = model.fit(
                x_train,
                y_train,
                epochs=epochs,
                validation_split=0.2,
                callbacks=[cp_callback],                
            )
            
            model.save(saved_dir)
            
            history_path_json = history_dir + model_name +"_history.json"
            json.dump(history.history, open(history_path_json, 'w'))
            
            history_path_csv = history_dir + model_name +"_history.csv"
            pd.DataFrame.from_dict(history.history).to_csv(history_path_csv, index=False)
            
            history_dict = history.history
            history_dict["epoch"] = [i+1 for i in range(len(history_dict["loss"]))]
            
            model_stats = pd.DataFrame(columns=stat_columns)
            model_stats = model_stats.append(pd.DataFrame(history_dict))
            
            model_stats["model_name"] = model_name
            model_stats["layers"] = layer_count
            model_stats["neurons"] = layer_neuron_count
            model_stats["activation"] = layer_activation
            
            session_stats = session_stats.append(model_stats)
            
            session_stats.to_csv(session_dir + session_name +"_stats.csv")
            


test1_L1_N16_sigmoid
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 16)                12560     
                                                                 
 dense_5 (Dense)             (None, 10)                170       
                                                                 
Total params: 12,730
Trainable params: 12,730
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
1471/1500 [============================>.] - ETA: 0s - loss: 0.8794 - accuracy: 0.8178
Epoch 1: saving model to ../../models/tf_colab_mnist/test1/test1_L1_N16_sigmoid/cp/cp-01.ckpt
1500/1500 [==============================] - 2s 1ms/step - loss: 0.8717 - accuracy: 0.8191 - val_loss

In [11]:
session_stats

,model_name,layers,neurons,activation,epoch,loss,accuracy,val_loss,val_accuracy
0,test1_L1_N16_sigmoid,1,16,sigmoid,1,0.864634,0.818438,0.422836,0.904833
1,test1_L1_N16_sigmoid,1,16,sigmoid,2,0.369443,0.905542,0.302149,0.921000
2,test1_L1_N16_sigmoid,1,16,sigmoid,3,0.293221,0.919458,0.262260,0.928167
0,test1_L1_N16_sigmoid,1,16,sigmoid,1,0.871720,0.819083,0.422318,0.907000
1,test1_L1_N16_sigmoid,1,16,sigmoid,2,0.365640,0.909604,0.305340,0.922417
2,test1_L1_N16_sigmoid,1,16,sigmoid,3,0.291929,0.921833,0.266054,0.925917
0,test1_L1_N16_relu,1,16,relu,1,0.504124,0.857292,0.278064,0.920250
1,test1_L1_N16_relu,1,16,relu,2,0.272842,0.921750,0.246548,0.928917
2,test1_L1_N16_relu,1,16,relu,3,0.240861,0.931229,0.236610,0.930750
0,test1_L1_N32_sigmoid,1,32,sigmoid,1,0.642683,0.863354,0.313939,0.918750
